In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None
import geopandas as gpd

C:\Users\llin\Anaconda3\lib\site-packages\geopandas\_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
C:\Users\llin\AppData\Local\Temp\ipykernel_24812\897408633.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/lat

## Project run

In [5]:
# read Project run loaded network
proj_raw = gpd.read_file(r'M:\Application\Model One\STIP2024\2050_TM152_STP_2024Project_00\OUTPUT\shapefile\network_links.shp')
proj = proj_raw[['CSPDEA', 'VMTEA',
                 'CSPDAM', 'VMTAM',
                 'CSPDMD', 'VMTMD',
                 'CSPDPM', 'VMTPM',
                 'CSPDEV', 'VMTEV',
                 'VMT24HR']]
display(proj.head())

,CSPDEA,VMTEA,CSPDAM,VMTAM,CSPDMD,VMTMD,CSPDPM,VMTPM,CSPDEV,VMTEV,VMT24HR
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# stack time periods

proj_all = pd.DataFrame(columns = ['CSPD', 'VMT', 'tod'])

for i in ['EA', 'AM', 'MD', 'PM', 'EV']:
    proj_part = proj[['CSPD'+i, 'VMT'+i]]
    proj_part.rename(columns = {'CSPD'+i: 'CSPD',
                                'VMT'+i:  'VMT'}, inplace=True)
    proj_part['tod'] = i
#     print(proj_part.shape[0])
    
    proj_all = pd.concat([proj_all, proj_part])

In [7]:
# add tags for congested conditions (speed less than 35 mph)
proj_all['cspd_cat'] = 'not_congested'
proj_all.loc[(proj_all.CSPD < 35) & (proj_all.CSPD != 0), 'cspd_cat'] = 'congested'

display(proj_all.head())

,CSPD,VMT,tod,cspd_cat
0,0.0,0.0,EA,not_congested
1,0.0,0.0,EA,not_congested
2,0.0,0.0,EA,not_congested
3,0.0,0.0,EA,not_congested
4,0.0,0.0,EA,not_congested


In [8]:
# calculate percent of VMT in congested conditions

vmt_tod_group_proj = proj_all.groupby(['tod', 'cspd_cat'])['VMT'].sum()
print('vmt by CSPD and time-period:')
display(vmt_tod_group_proj.unstack())

vmt_group_proj = proj_all.groupby('cspd_cat')['VMT'].sum().reset_index()
print('total vmt by CSPD:')
display(vmt_group_proj)

vmt by CSPD and time-period:


cspd_cat,congested,not_congested
tod,,
AM,2.243668e+07,2.662236e+07
EA,1.697557e+06,6.163432e+06
EV,1.016466e+07,2.256228e+07
MD,1.913407e+07,3.302238e+07
PM,2.543854e+07,2.890376e+07


total vmt by CSPD:


,cspd_cat,VMT
0,congested,7.887151e+07
1,not_congested,1.172742e+08


## No Project Run

In [10]:
# read NoProject run loaded network
noProj_raw = gpd.read_file(r'M:\Application\Model One\STIP2024\2050_TM152_STP_2024NoProject_00\OUTPUT\shapefile\network_links.shp')
noProj = noProj_raw[['CSPDEA', 'VMTEA',
                     'CSPDAM', 'VMTAM',
                     'CSPDMD', 'VMTMD',
                     'CSPDPM', 'VMTPM',
                     'CSPDEV', 'VMTEV',
                     'VMT24HR']]
display(noProj.head())

,CSPDEA,VMTEA,CSPDAM,VMTAM,CSPDMD,VMTMD,CSPDPM,VMTPM,CSPDEV,VMTEV,VMT24HR
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
# stack time periods

noProj_all = pd.DataFrame(columns = ['CSPD', 'VMT', 'tod'])

for i in ['EA', 'AM', 'MD', 'PM', 'EV']:
    noProj_part = noProj[['CSPD'+i, 'VMT'+i]]
    noProj_part.rename(columns = {'CSPD'+i: 'CSPD',
                                  'VMT'+i:  'VMT'}, inplace=True)
    noProj_part['tod'] = i
#     print(noProj_part.shape[0])
    
    noProj_all = pd.concat([noProj_all, noProj_part])

In [12]:
# add tags for congested conditions (speed less than 35 mph)
noProj_all['cspd_cat'] = 'not_congested'
noProj_all.loc[(noProj_all.CSPD < 35) & (noProj_all.CSPD != 0), 'cspd_cat'] = 'congested'

display(noProj_all.head())

,CSPD,VMT,tod,cspd_cat
0,0.0,0.0,EA,not_congested
1,0.0,0.0,EA,not_congested
2,0.0,0.0,EA,not_congested
3,0.0,0.0,EA,not_congested
4,0.0,0.0,EA,not_congested


In [13]:
# calculate percent of VMT in congested conditions

vmt_tod_group_noProj = noProj_all.groupby(['tod', 'cspd_cat'])['VMT'].sum()
print('vmt by CSPD and time-period:')
display(vmt_tod_group_noProj.unstack())

vmt_group_noProj = noProj_all.groupby('cspd_cat')['VMT'].sum().reset_index()
print('total vmt by CSPD:')
display(vmt_group_noProj)

vmt by CSPD and time-period:


cspd_cat,congested,not_congested
tod,,
AM,2.282263e+07,2.609980e+07
EA,1.707818e+06,6.199477e+06
EV,1.019245e+07,2.251193e+07
MD,1.918133e+07,3.293587e+07
PM,2.586790e+07,2.831587e+07


total vmt by CSPD:


,cspd_cat,VMT
0,congested,7.977214e+07
1,not_congested,1.160629e+08


## Summary

In [14]:
pct_proj = vmt_group_proj.VMT.values[0] / vmt_group_proj.VMT.sum()
print('{} of VMT is in congested conditions for Project run'.format(pct_proj))

pct_noProj = vmt_group_noProj.VMT.values[0] / vmt_group_noProj.VMT.sum()
print('{} of VMT is in congested conditions for NoProject run'.format(pct_noProj))

0.4021066794328571 of VMT is in congested conditions for Project run
0.4073434520515617 of VMT is in congested conditions for NoProject run
